## Setup iniciais


### Importando as bibliotecas necesarias

In [1]:
import pandas as pd
import requests
import plotly.express as px
from pandas import json_normalize

### Alterando as definicoes de exibicao das tableas

In [2]:
pd.set_option('display.max_columns', None)

### Pegando o requeste do endereco, se a resposta for 200 recebos o valor e colocamos como o df usando o pandas, se nao exibimos messagem de erro

In [3]:
response = requests.get("https://challenge-data-science-3ed.s3.amazonaws.com/Telco-Customer-Churn.json")

if response.status_code == 200:
  data = response.json()
  df = pd.DataFrame(data)
else:
  print("erro ao caregar")

## Ajustando as tabelas

### Salvando os json de dentro da tabela em variaveis e depois removendo elas da tabela e colocando novas colunas

In [4]:
customer = json_normalize(df['customer'])
phone = json_normalize(df['phone'])
internet = json_normalize(df['internet'])
account = json_normalize(df['account'])
df.drop(columns=['customer', 'phone', 'internet', 'account'], inplace = True)
df = df.join(customer, rsuffix='_customer')
df = df.join(phone, rsuffix='_phone')
df = df.join(internet, rsuffix='_internet')
df = df.join(account, rsuffix='_account')

### Verificar se tem algum NaN na tabela

In [5]:
df.isna().sum()

customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Charges.Monthly     0
Charges.Total       0
dtype: int64

### Verificar se tem algum null na tabela

In [6]:
df.isnull().sum()

customerID          0
Churn               0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
Charges.Monthly     0
Charges.Total       0
dtype: int64

### Substituindo Yes e No por 1 e 0

In [7]:
df.replace({'Yes':1, 'No':0}, inplace=True)

###Substituindo o tipo de serviso por se a pessoa tem fibra ou nao

In [8]:
df['InternetService'].unique()


array(['DSL', 'Fiber optic', 0], dtype=object)

In [9]:
df['FiberOptic'] = df['InternetService'].replace({"Fiber optic": 1, "DSL":0, 0:0})
df.drop(columns=["InternetService"], inplace = True)

###Substituindo o tempo de contrado de sting por float

In [10]:
df["Contract"].replace({"One year": 0.5, "Two year": 1, "Month-to-month": 0.042}, inplace = True)

### Criando colunas para cada metodo de pagamento

In [11]:
df['Payment:MailedCheck'] = df['PaymentMethod'].apply(lambda x: 1 if x == "Mailed check" else 0)
df['Payment:Electroniccheck'] = df['PaymentMethod'].apply(lambda x: 1 if x == "Electronic check" else 0)
df['Payment:CreditCard(automatic)'] = df['PaymentMethod'].apply(lambda x: 1 if x == "Credit card (automatic)" else 0)
df['Payment:BankTransfer(automatic)'] = df['PaymentMethod'].apply(lambda x: 1 if x == "Bank transfer (automatic)" else 0)
df.drop(columns=['PaymentMethod'], inplace = True)

### Substituindo a coluna gender por uma coluna Male contendo 0 e 1

In [12]:
df['Male'] = df['gender'].replace({"Male": 1, "Female":0})

In [13]:
df.drop(columns=['gender'], inplace = True)
df.drop(columns=["customerID"], inplace = True)

### Retirando a redundacia do No internet service e do No phone service

In [14]:
df.replace({'No internet service': 0, 'No phone service': 0}, inplace = True)

### Normalizando as colunas Charges Total, Charges Monthly e tenure

### Convertendo a Charges Total para numerico

In [15]:
df['Charges.Total'] = pd.to_numeric(df['Charges.Total'], errors='coerce')

In [16]:
df['Charges.Monthly'] = (df['Charges.Monthly'] - df['Charges.Monthly'].min()) / (df['Charges.Monthly'].max() - df['Charges.Monthly'].min())
df['Charges.Total'] = (df['Charges.Total'] - df['Charges.Total'].min()) / (df['Charges.Total'].max() - df['Charges.Total'].min())
df['tenure'] = (df['tenure'] - df['tenure'].min()) / (df['tenure'].max() - df['tenure'].min())

In [17]:
df.rename(columns={"Churn":"DeixouAEmpresa", "SeniorCitizen":"CidadaoSenior", "Partner":"Parceiro", "Dependents":"Dependentes", "tenure":"TempoDeContrato", "PhoneService":"ServisoTelefonico", "MultipleLines":"MultiplasLinhas", "OnlineSecurity":"SegurancaOnline", "OnlineBackup":"BackupOnline", "DeviceProtection":"ProtecaoDeDispositivo", "TechSupport":"SuporteTecnico", "StreamingTV":"ServisoDeStreamingTv", "StreamingMovies":"ServisoDeStreamingFilmes", "Contract":"TipoDeContrato", "PaperlessBilling":"FaturaOnline", "Charges.Monthly":"ValorMensal", "Charges.Total":"ValorTotal", "FiberOptic":"FibraOptica", "Payment:MailedCheck":"Pagamento:ChequePorCorreios", "Payment:Electroniccheck":"Pagamento:ChequeEletronico", "Payment:CreditCard(automatic)":"Pagamento:CartaoDeCredito", "Payment:BankTransfer(automatic)":"Pagamento:TransferenciaBancaria", "Male":"Homen"}, inplace=True)

In [18]:
df_empty_strings = df[df['DeixouAEmpresa'] == '']
df = df[df['DeixouAEmpresa'] != '']
df['DeixouAEmpresa'] = df['DeixouAEmpresa'].astype('float64')  # ou 'int64' dependendo dos seus dados



<ipython-input-18-6d2b5a72c459>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DeixouAEmpresa'] = df['DeixouAEmpresa'].astype('float64')  # ou 'int64' dependendo dos seus dados


In [19]:
grouped_df = df.groupby(['CidadaoSenior', 'DeixouAEmpresa']).size().reset_index(name='NumeroDePessoas')

fig = px.bar(grouped_df,
             x='CidadaoSenior',
             y='NumeroDePessoas',
             color='DeixouAEmpresa',
             title='Relação entre Cidadão Sênior e Deixou a Empresa',
             labels={'CidadaoSenior': 'Cidadão Sênior', 'NumeroDePessoas': 'Número de Pessoas'})

fig.show()

In [20]:
fig = px.box(df, x='DeixouAEmpresa', y='ValorMensal', title='Relação entre Deixou a Empresa e Valor Mensal')
fig.show()


In [21]:
corr_matrix = df.corr()

fig = px.imshow(corr_matrix,
                labels=dict(x="Variáveis", y="Variáveis", color="Correlação"),
                x=corr_matrix.columns,
                y=corr_matrix.columns,
                title='Matriz de Correlação')

fig.show()
